13519093 - Wisnu Aditya Samiadji

13519169 - David Owen Adiwiguna

Tugas 2 Kriptografi

DAWASCrypt

In [ ]:
import hashlib
import random
from math import exp, expm1
import string
import time

In [ ]:
def generateuniquekey():
  return ''.join(random.choices(string.ascii_lowercase + string.digits, k=36))

In [ ]:
ROUNDS = 16
BLOCKSIZE = 16
BLOCKSIZE_BITS = 128
UNIQUE = generateuniquekey() #Run this only once

In [ ]:
def shuffle(message, key):
    random.seed(key)
    l = list(range(len(message)))
    random.shuffle(l)
    return [message[x] for x in l]

def unshuffle(shuffled_message, key):
    random.seed(key)
    l = list(range(len(shuffled_message)))
    random.shuffle(l)
    out = [None] * len(shuffled_message)
    for i, x in enumerate(l):
        out[x] = shuffled_message[i]
    return out

In [ ]:
def encrypt(key, message, mode):
    ciphertext = ""
    n = BLOCKSIZE  
    
    message = [message[i: i + n] for i in range(0, len(message), n)]

    lastBlockLength = len(message[len(message)-1])

    if ( lastBlockLength < BLOCKSIZE):
        for i in range(lastBlockLength, BLOCKSIZE):
            message[len(message)-1] += " "


    # generate a 16 bit key based of user inputted key
    #print("md5key for", key, "is", key_md5(key))
    key = key_md5(key)
    key_initial = key
    ctr = 0
    for block in message:
        #print("Encrypt",key)
        #generate sbox based on md5key
        sbox = generatesbox(key)
        L = [""] * (ROUNDS + 1)
        R = [""] * (ROUNDS + 1)
        L[0] = block[0:BLOCKSIZE//2]
        R[0] = block[BLOCKSIZE//2:BLOCKSIZE]

        for i in range(1, ROUNDS+1):
            round_key = subkeygen(str(i), key, i)
            LR_im = R[i - 1][:BLOCKSIZE//4]
            RR_im = R[i - 1][BLOCKSIZE//4:]
  
            LL_i = RR_im
            RL_i = xor(LR_im, transform(RR_im, i, round_key, sbox))

            L[i] = LL_i + RL_i
            R[i] = xor(L[i - 1], transform(R[i - 1], i, round_key, sbox))

        partial_cipher = L[ROUNDS] + R[ROUNDS]
        shuffle(partial_cipher, key)
        ciphertext += partial_cipher
        if (mode == "cbc"):
            key = subkeygen(L[0], key, i)
        if (mode == "counter"):
            key = subkeygen(str(ctr), key_initial, i)
            ctr += 1

    return ciphertext


In [ ]:
def decrypt(key, ciphertext, mode):
    message = ""
    n = BLOCKSIZE  
    
    ciphertext = [ciphertext[i: i + n] for i in range(0, len(ciphertext), n)]

    lastBlockLength = len(ciphertext[len(ciphertext)-1])

    if ( lastBlockLength < BLOCKSIZE):
        for i in range(lastBlockLength, BLOCKSIZE):
            ciphertext[len(ciphertext)-1] += " "


    # generate a 128 bit key based off the user inputted key using md5
    key = key_md5(key)
    
    
    key_initial = key
    ctr = 0
    for block in ciphertext:
        #print("Decrypt",key)
        #generate sbox based on md5key
        sbox = generatesbox(key)
        #print ("Block: " + block)
        L = [""] * (ROUNDS + 1)
        R = [""] * (ROUNDS + 1)
        L[ROUNDS] = block[0:BLOCKSIZE//2]
        R[ROUNDS] = block[BLOCKSIZE//2:BLOCKSIZE]

        for i in range(ROUNDS, 0, -1):
            round_key = subkeygen(str(i), key, i)
            LL_i = L[i][:BLOCKSIZE//4]
            RL_i = L[i][BLOCKSIZE//4:]

            RR_im = LL_i
            LR_im = xor(RL_i, transform(RR_im, i, round_key, sbox))

            R[i - 1] = LR_im + RR_im
            L[i - 1] = xor(R[i], transform(R[i - 1], i, round_key, sbox))

        partial_message = L[0] + R[0]
        unshuffle(partial_message, key)
        message += partial_message
        if (mode == "cbc"):
            key = subkeygen(L[0], key, i)
            # print(key)
        if (mode == "counter"):
            key = subkeygen(str(ctr), key_initial, i)
            ctr += 1
            # print(key)

    return message

In [ ]:
def key_md5(key):
    return hashlib.md5((key+UNIQUE).encode('utf-8')).hexdigest()
def subkeygen(s1, s2, i):
    result = hashlib.md5((s1+s2).encode('utf-8')).hexdigest()
    return result

In [ ]:
#fungsi untuk melakukan transformasi
def transform(x, i, k, sbox):
    k = stobin(k)
    x = stobin(str(x))
    if (len(x) == 32) :
        out = ""
        for i in range(8):
            val = bintoint(x[i*4:(i*4) + 4])
            out += bin(sbox[i].index(val))[2:].zfill(4)

        out = out[4:len(out)] + out[0:4]
    else:
        out = x
    k = bintoint(k)
    x = bintoint(out)

    res = pow((x * k), i)
    res = itobin(res)
    return bintostr(res)

In [ ]:
# xor two strings
def xor(s1, s2):
  return ''.join(chr(ord(a) ^ ord(b)) for a, b in zip(s1, s2))


# string to binary with length 8
def stobin(s):
  return ''.join('{:08b}'.format(ord(c)) for c in s)


# binary to int
def bintoint(s):
    return int(str(s), 2)


# int to binary
def itobin(i):
    return bin(i)


# binary to string
def bintostr(b):
    n = int(b, 2)
    return ''.join(chr(int(b[i: i + 8], 2)) for i in range(0, len(b), 8))

#hexadecimal to decimal
def hextodec(s):
    return int(str(s), 16)


In [ ]:
def generatesbox(key):
  sb1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  sb2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  sb3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  sb4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  sb5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  sb6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  sb7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  sb8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  for i in range (1,9):
    for j in range (1,17):
      if(i == 1):
        hdec1 = (hextodec(key[(j - 1) * 2 : (j - 1) * 2 + 2]) + i + j) % 16
        sb1 = sb1[hdec1:] + sb1[:hdec1]
      if(i == 2):
        hdec2 = (hextodec(key[(j - 1) * 2 : (j - 1) * 2 + 2]) + i + j) % 16
        sb2 = sb2[-hdec2:] + sb2[:-hdec2]
      if(i == 3):
        hdec3 = (hextodec(key[(j - 1) * 2 : (j - 1) * 2 + 2]) + i + j) % 16
        sb3 = sb3[hdec3:] + sb3[:hdec3]
      if(i == 4):
        hdec4 = (hextodec(key[(j - 1) * 2 : (j - 1) * 2 + 2]) + i + j) % 16
        sb4 = sb4[-hdec4:] + sb4[:-hdec4]
      if(i == 5):
        hdec5 = (hextodec(key[(j - 1) * 2 : (j - 1) * 2 + 2]) + i + j) % 16
        sb5 = sb5[hdec5:] + sb5[:hdec5]
      if(i == 6):
        hdec6 = (hextodec(key[(j - 1) * 2 : (j - 1) * 2 + 2]) + i + j) % 16
        sb6 = sb6[-hdec6:] + sb6[:-hdec6]
      if(i == 7):
        hdec7 = (hextodec(key[(j - 1) * 2 : (j - 1) * 2 + 2]) + i + j) % 16
        sb7 = sb7[hdec7:] + sb7[:hdec7]
      if(i == 8):
        hdec8 = (hextodec(key[(j - 1) * 2 : (j - 1) * 2 + 2]) + i + j) % 16
        sb8 = sb8[-hdec8:] + sb8[:-hdec8]
  sb = [ sb1, sb2, sb3, sb4, sb5, sb6, sb7, sb8 ]
  return sb

In [ ]:
def determineMode(inp):
  if(inp % 3 == 0):  return "cbc"
  elif (inp % 3 == 1): return "ecb"
  else: return "counter"

Plaintext 1

"Theories about learning with multimedia can be positioned at different levels. At a basic level, psychological theories describe memory systems and cognitive processes that explain how people process different types of information and how they learn with different senses."

In [ ]:
# plaintextinp1 = str(input("Plaintext: "))
# keyinp1 = str(input("Key: "))
# mode1 = determineMode(int(input("Mode (1:ecb, 2:counter, 3: cbc): ")))
plaintextinp1 = "Theories about learning with multimedia can be positioned at different levels. At a basic level, psychological theories describe memory systems and cognitive processes that explain how people process different types of information and how they learn with different senses."
keyinp1 = "ameagari no niji ma"
mode1 = determineMode(1)
st = time.time()
ciptext1 = encrypt(keyinp1, plaintextinp1, mode1)
print(ciptext1)
et = time.time()
encrypt_time = et - st
print('Encryption time:', encrypt_time, 'seconds')
st = time.time()
plaintext1 = decrypt(keyinp1, ciptext1, mode1)
print(plaintext1)
dt = time.time()
decrypt_time = dt - st
print('Decryption time:', decrypt_time, 'seconds')

n -Ï¦ýÁÃ¶0C-GByk­>ÐÆÃ®jÎÅÙÃ$+ÒÞª¨oëKÿÙ+YSU¨@fEÆéo)ÉDcoÀ×1ÜÊÙ#ãUõH²fï50'	Ô#T-}YnOîçëæÌV#OÌ±S¿ÎÃ²~ÕÄ,þµöÝ*(8¬ÖµÒ~Î¼0<Ê>='§Ï~öIF=ÿ_Å#M=c-ü®ôÇ4lãL°ÞíÞö¹7]là/'ýø/ÔÓ]jÜÏEcïã-&ÈUnQ°Ëí+¿P:!!|Eü]LV É&8­þ/z{
Encryption time: 0.23798084259033203 seconds
Theories about learning with multimedia can be positioned at different levels. At a basic level, psychological theories describe memory systems and cognitive processes that explain how people process different types of information and how they learn with different senses.
Decryption time: 0.16456961631774902 seconds


In [ ]:
# plaintextinp2 = str(input("Plaintext: "))
# keyinp2 = str(input("Key: "))
# mode2 = determineMode(int(input("Mode (1:ecb, 2:counter, 3:cbc): ")))
plaintextinp2 = "Theories about learning with multimedia can be positioned at different levels. At a basic level, psychological theories describe memory systems and cognitive processes that explain how people process different types of information and how they learn with different senses."
keyinp2 = "ameagari no niji ma"
mode2 = determineMode(2)
st = time.time()
ciptext2 = encrypt(keyinp2, plaintextinp2, mode2) 
print(ciptext2)
et = time.time()
encrypt_time = et - st
print('Encryption time:', encrypt_time, 'seconds')
st = time.time()
plaintext2 = decrypt(keyinp2, ciptext2, mode2)
print(plaintext2)
dt = time.time()
decrypt_time = dt - st
print('Decryption time:', decrypt_time, 'seconds')

8ªÍUG Sùl
Ã0üÞÜ\nf.DvÑ?&dÇámÈ8,Ék³/Rª	´hË Ï	"ó{FNbÿHQã
Encryption time: 0.16413474082946777 seconds
Theories about learning with multimedia can be positioned at different levels. At a basic level, psychological theories describe memory systems and cognitive processes that explain how people process different types of information and how they learn with different senses.
Decryption time: 0.15146470069885254 seconds


In [ ]:
# plaintextinp3 = str(input("Plaintext: "))
# keyinp3 = str(input("Key: "))
# mode3 = determineMode(int(input("Mode (1:ecb, 2:counter, 3:cbc): ")))
plaintextinp3 = "Theories about learning with multimedia can be positioned at different levels. At a basic level, psychological theories describe memory systems and cognitive processes that explain how people process different types of information and how they learn with different senses."
keyinp3 = "ameagari no niji ma"
mode3 = determineMode(3)
st = time.time()
ciptext3 = encrypt(keyinp3, plaintextinp3, mode3)
print(ciptext3)
et = time.time()
encrypt_time = et - st
print('Encryption time:', encrypt_time, 'seconds')
st = time.time()
plaintext3 = decrypt(keyinp3, ciptext3, mode3)
print(plaintext3)
dt = time.time()
decrypt_time = dt - st
print('Decryption time:', decrypt_time, 'seconds')
print(plaintext1 == plaintext2 and plaintext1 == plaintext3)
print(ciptext1 == ciptext2 or ciptext1 == ciptext3 or ciptext2 == ciptext3)

kî-ÓcËPá#¦0´TB'}cg6y¡v
Encryption time: 0.15140485763549805 seconds
Theories about learning with multimedia can be positioned at different levels. At a basic level, psychological theories describe memory systems and cognitive processes that explain how people process different types of information and how they learn with different senses.
Decryption time: 0.1523897647857666 seconds
True
False


In [ ]:
# plaintextinp2 = str(input("Plaintext: "))
# keyinp2 = str(input("Key: "))
# mode2 = determineMode(int(input("Mode (1:ecb, 2:counter, 3:cbc): ")))
plaintextinp2 = "Theories about learning with multimedia can be positioned at different levels. At a basic level, psychological theories describe memory systems and cognitive processes that explain how people process different types of information and how they learn with different senses."
keyinp2 = "ameagari no niji ma"
mode2 = determineMode(2)
st = time.time()
ciptext2 = encrypt(keyinp2, plaintextinp2 + '.', mode2)
print(ciptext2)
et = time.time()
encrypt_time = et - st
print('Encryption time:', encrypt_time, 'seconds')
st = time.time()
plaintext2 = decrypt(keyinp2, ciptext2, mode2)
print(plaintext2)
dt = time.time()
decrypt_time = dt - st
print('Decryption time:', decrypt_time, 'seconds')

8ªÍUG Sùl
Ã0üÞÜ\nf.DvÑ?&dÇámÈ8,Ék³/Rª	´hË Ï	"ó{FNbÿHQãsFÉ(yAê]è*]òÀ
Encryption time: 0.16357111930847168 seconds
Theories about learning with multimedia can be positioned at different levels. At a basic level, psychological theories describe memory systems and cognitive processes that explain how people process different types of information and how they learn with different senses..               
Decryption time: 0.1736738681793213 seconds


Plaintext 2

if saying "I won't let you run away," this made Amane feel a little shy,
and then he put his hands behind Maui and hugged her back.
I won't run...
I want to treat her well, I want her to be happy, I want to love her. These
thoughts flew through Amane's mind as he embraced her.
"I wish for Mahiru to be happy.

In [ ]:
# plaintextinp1 = str(input("Plaintext: "))
# keyinp1 = str(input("Key: "))
# mode1 = determineMode(int(input("Mode (1:ecb, 2:counter, 3: cbc): ")))
plaintextinp1 = """if saying "I won't let you run away," this made Amane feel a little shy,
and then he put his hands behind Maui and hugged her back.
I won't run...
I want to treat her well, I want her to be happy, I want to love her. These
thoughts flew through Amane's mind as he embraced her.
"I wish for Mahiru to be happy."""
keyinp1 = "daisuko da yo"
mode1 = determineMode(1)
st = time.time()
ciptext1 = encrypt(keyinp1, plaintextinp1, mode1)
print(ciptext1)
et = time.time()
encrypt_time = et - st
print('Encryption time:', encrypt_time, 'seconds')
st = time.time()
plaintext1 = decrypt(keyinp1, ciptext1, mode1)
print(plaintext1)
dt = time.time()
decrypt_time = dt - st
print('Decryption time:', decrypt_time, 'seconds')

¤ÚÀtËÕ$öµäíúTç³n¾ÛR³"s)ün÷í<jQ[j¿r£UùäËY_Í«D/H¡ëNÜn+¤ªx±
Encryption time: 0.18240666389465332 seconds
if saying "I won't let you run away," this made Amane feel a little shy,
and then he put his hands behind Maui and hugged her back.
I won't run...
I want to treat her well, I want her to be happy, I want to love her. These
thoughts flew through Amane's mind as he embraced her.
"I wish for Mahiru to be happy.           
Decryption time: 0.196197509765625 seconds


In [ ]:
# plaintextinp2 = str(input("Plaintext: "))
# keyinp2 = str(input("Key: "))
# mode2 = determineMode(int(input("Mode (1:ecb, 2:counter, 3:cbc): ")))
plaintextinp2 ="""if saying "I won't let you run away," this made Amane feel a little shy,
and then he put his hands behind Maui and hugged her back.
I won't run...
I want to treat her well, I want her to be happy, I want to love her. These
thoughts flew through Amane's mind as he embraced her.
"I wish for Mahiru to be happy."""
keyinp2 = "daisuko da yo"
mode2 = determineMode(2)
st = time.time()
ciptext2 = encrypt(keyinp2, plaintextinp2, mode2)
print(ciptext2)
et = time.time()
encrypt_time = et - st
print('Encryption time:', encrypt_time, 'seconds')
st = time.time()
plaintext2 = decrypt(keyinp2, ciptext2, mode2)
print(plaintext2)
dt = time.time()
decrypt_time = dt - st
print('Decryption time:', decrypt_time, 'seconds')

¼þÅ¦	yÇÛ7îÊüUâó7I5Õü!fJ×©OÚ=6Rs·H0E®þ î%Ö_k6üÏµMNïNö1åÏo2STÙuxaÇ5=
Ã ÂµÝ¢¶U}^$Õýù±jÑHvµ5	3]ofz9`¡å~Ú
V*ZúF|¿ù«¨õÔBN_DxõïU¦uSìjm¼µëù%*x(°%w¦%9=J<W
Encryption time: 0.19705629348754883 seconds
if saying "I won't let you run away," this made Amane feel a little shy,
and then he put his hands behind Maui and hugged her back.
I won't run...
I want to treat her well, I want her to be happy, I want to love her. These
thoughts flew through Amane's mind as he embraced her.
"I wish for Mahiru to be happy.           
Decryption time: 0.18267154693603516 seconds


In [ ]:
# plaintextinp3 = str(input("Plaintext: "))
# keyinp3 = str(input("Key: "))
# mode3 = determineMode(int(input("Mode (1:ecb, 2:counter, 3:cbc): ")))
plaintextinp3 = """if saying "I won't let you run away," this made Amane feel a little shy,
and then he put his hands behind Maui and hugged her back.
I won't run...
I want to treat her well, I want her to be happy, I want to love her. These
thoughts flew through Amane's mind as he embraced her.
"I wish for Mahiru to be happy."""
keyinp3 = "daisuko da yo"
mode3 = determineMode(3)
st = time.time()
ciptext3 = encrypt(keyinp3, plaintextinp3, mode3)
print(ciptext3)
et = time.time()
encrypt_time = et - st
print('Encryption time:', encrypt_time, 'seconds')
st = time.time()
plaintext3 = decrypt(keyinp3, ciptext3, mode3)
print(plaintext3)
dt = time.time()
decrypt_time = dt - st
print('Decryption time:', decrypt_time, 'seconds')
print(plaintext1 == plaintext2 and plaintext1 == plaintext3)
print(ciptext1 == ciptext2 or ciptext1 == ciptext3 or ciptext2 == ciptext3)

¼þÅ¦	yÇÛ7îÊb­±ç¡ïøiÜh8[5Æïô}ðPQQvÊÉ³a?,*û<zÀ9ë6Zc·®èio]
RâUÅEà]k+YÒ
 ÓþM`g©¬*>5Oéüê¸ÃQ²a_µ>rÝS¸xË_oò×kÑóâ_æ·ª+×úeB0R j.Gâ.²lÍ¢¨êuÏóà.fA]	âÒ®w¯|t7»¼b{­ÎÞy»T·2	<8PIæ'm¿mAí0m6
©2÷è5ÉóRÕ>"ÛÓ/¿RäEîåYà~G½l1ÊÚ57­Öò¸¼_äUTÄNàÎ,þ	2ÊßPíìªw¨:æUGç¥ûVLqâ¦
Encryption time: 0.1867833137512207 seconds
if saying "I won't let you run away," this made Amane feel a little shy,
and then he put his hands behind Maui and hugged her back.
I won't run...
I want to treat her well, I want her to be happy, I want to love her. These
thoughts flew through Amane's mind as he embraced her.
"I wish for Mahiru to be happy.           
Decryption time: 0.17600369453430176 seconds
True
False


In [ ]:
# plaintextinp1 = str(input("Plaintext: "))
# keyinp1 = str(input("Key: "))
# mode1 = determineMode(int(input("Mode (1:ecb, 2:counter, 3: cbc): ")))
plaintextinp1 = """if saying "I won't let you run away," this made Amane feel a little shy,
and then he put his hands behind Maui and hugged her back.
I won't run...
I want to treat her well, I want her to be happy, I want to love her. These
thoughts flew through Amane's mind as he embraced her.
"I wish for Mahiru to be happy."""
keyinp1 = "daisuko da yo"
mode1 = determineMode(1)
st = time.time()
ciptext1 = encrypt(keyinp1, plaintextinp1, mode1)
print(ciptext1)
et = time.time()
encrypt_time = et - st
print('Encryption time:', encrypt_time, 'seconds')
st = time.time()
plaintext1 = decrypt("daisuki da yo", ciptext1, mode1)
print(plaintext1)
dt = time.time()
decrypt_time = dt - st
print('Decryption time:', decrypt_time, 'seconds')

µWìÊ¨( CcÔß1ÊZ»/tV;!(q»Vflþ@~ï§¼àôøYõÒÊ§04Tù®ÖqàÛª`ÖÇjXÖ²åÞýq/ðbùàe¸W°mãWÛÔiJdN#ÿí °Å/ÍÒPÔfûiVÎ=
ãÍX
Encryption time: 0.19034314155578613 seconds
wzAñá!áaÈ¿JáÎÍÂôuC@>K]ÃÞ,eÑc·Vå+çfæ-3øq ]náw;^ê=ïËo[ÀrÍÅaF*½^chæqýAGÎ³è¡ááÈúÊ[äË]¾[x×&öN-»ñ þ
ézñwòonÐï²±ÑáÂ8oÏeÏh£V¸«YêI»þø°¨2|#[7jiyrú­xAóÕ¡¿¢'z_îØðn&4Ï+ÙÄìvHç§t¹á÷øÇåTÈa×cî?¤ÏD0xÐ_)ÑÛ1]ÃCZ
Decryption time: 0.18317651748657227 seconds
